# INF221 Assignment 3
Jon Nordby

In [1]:
%matplotlib inline

import math
import random

import matplotlib
import pandas

## Exercise 10

CLRS 4.1-1: What does Find-Maximum-Subarray return when all elements of A are negative?


zero / empty-set ? 

## Exercise 11

CLRS 4.1-3 (partial):

Implement both the brute-force and the recursive algorithms for the maximum-subarray problem on your own computer using Python. What problem size $n_0$ gives the crossover point at which the recursive algorithm beats the brute force algorithm?

Hints:

Use Python module timeit to time execution of your code. 
To create input data, use cumulative sums of random numbers as in the example towards the end of Lecture 4.


In [2]:
# page 68-74
def maximum_subarray_bruteforce(A):
    pass


def maximum_subarray_recursive(A):
    pass



## Exercise 12

CLRS 4.3-1 & 2:

a) Show that the solution of $T(n)=T(n-1)+n$ is $O(n^2)$.

b) Show that the solution of $T(n) = T(\lceil n/2 \rceil)+1$ is $O(\lg n)$.

 Recursion
 Substitution method?
 See Assignment 2, Ex 7

## Exercise 13
CLRS 4.3-3

We saw that the solution of $T(n)=2T(\lfloor n/2 \rfloor)+n$ is $O(n\lg n)$.
Show that the solution of this recurrence is also: $\Omega(n\lg n)$ and conclude that the solution is $\Theta(n\lg n)$.

Hint: Use the same approach as for the upper bound in Ch 4.3. You will then need to find at some point an expression that is smaller than $\lfloor n/2 \rfloor \lg(\lfloor n/2 \rfloor)$. Here, it can be useful to use different lower bounds for $\lfloor n/2 \rfloor$ outside and inside the logarithm.


## Exercise 14
CLRS 4.4-1, 5:

Use recursion trees to determine a good asymptotic upper bound on the recurrences given below and then use the substitution method to verify your answers.

Note: This exercise can give you up to two points extra credit. You receive one point each for solving a) and b), and a third point for a tight bound in b).

a) $T(n) = 3T(\lfloor n/2\rfloor)+n$

Hint: The recursion tree analysis analysis is similar to the first example in Ch 4.4. For the proof using the substitution method, also take a look at the "Subtleties" section of Ch 4.3.

b) $T(n) = T(n-1)+T(n/2) + n$

Hint: To obtain an upper bound, perform a "worst case" analysis along the left edge of the recursion tree and assume that all levels are full. Prove that bound using the substitution method. For extra credit, attempt at tighter bound!

Page 86-90

## Exercise 15
CLRS 4.5-1:

Use the master method to give tight asymptotic bounds for the following recurrences:

### a) $T(n) = 2T(n/4)+1$

The equation is on the form required by Master method: $T(n) = aT(n/b) + f(n)$ with $a=2, b=4, f(n)=1$

$\Theta(f(n)) = \Theta(1) = \Theta(n^0)$ follows Case 1 (asymptotic runtime dominated by tree traversal), $\Theta(n^{log_b{a-e}})$ with $e=1$: $\Theta(n^{\log_4{2-1}}) = \Theta(1)$.

Thus the solution is $T(n) = \Theta(n^{\log_b a}) = \Theta(n^{\log_4 2}) = \Theta(n^{0.5}) = \Theta(\sqrt{n})$

### b) $T(n) = 2T(n/4) + \sqrt{n}$

Again $a=2, b=4$ and $f(n) = \sqrt{n}$.

$\Theta(f(n)) = \Theta(\sqrt{n}) = \Theta(n^{0.5})$ follows Case 2 (asymptotic runtime a combination of tree traversal and leaf function execution), $\Theta(n^{log_b{a}})$: $\Theta(n^{\log_4{2}}) = \Theta(\sqrt{n})$.

The solution for $T(n) = \Theta(f(n^{\log_b a} \lg n) = \Theta(n^{\log_4 2} \lg n) = \Theta(\sqrt{n}\lg n)$ 

### c) $T(n) = 2T(n/4)+n$

$a=2, b=5, f(n)=n$

$\Theta(f(n)) = \Theta(n) = \Theta(n^{1})$ follows Case 3 (asymptotic runtime dominated by leaf function execution) with $e=2$, $\Theta(n^{log_b{a+e}})$: $\Theta(n^{\log_4{2+2}}) = \Theta(n)$.

Therefore $T(n) = \Theta(f(n)) = \Theta(n)$ 

### d) $T(n) = 2T(n/4)+n^2$

$a=2, b=4, f(n) = n^2$

$\Theta(f(n)) = \Theta(n^2)$ follows Case 3 of the method with $e=14$, $\Theta(n^{log_b{a+e}})$: $\Theta(n^{\log_4{2+14}}) = \Theta(n^2)$.

Therefore $T(n) = \Theta(f(n)) = \Theta(n^2)$